In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

# 한파

In [36]:
# 데이터 로드

# 파일 경로 설정
path = './data/한파/S-DoT_NATURE_*.csv'
files = glob.glob(path)
df_list = [pd.read_csv(f, encoding='cp949') for f in files]
cold_wave_df = pd.concat(df_list, ignore_index=True)


/var/folders/zh/lv2frb2s6_qbjc1lcfgty_th0000gn/T/ipykernel_21211/2351571185.py:6: DtypeWarning: Columns (0: 기온(℃), 1: 자외선(UVI)) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(f, encoding='cp949') for f in files]
/var/folders/zh/lv2frb2s6_qbjc1lcfgty_th0000gn/T/ipykernel_21211/2351571185.py:6: DtypeWarning: Columns (0: 흑구 운도(℃)) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(f, encoding='cp949') for f in files]
/var/folders/zh/lv2frb2s6_qbjc1lcfgty_th0000gn/T/ipykernel_21211/2351571185.py:6: DtypeWarning: Columns (0: 기온(℃), 1: 흑구 운도(℃)) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(f, encoding='cp949') for f in files]
/var/folders/zh/lv2frb2s6_qbjc1lcfgty_th0000gn/T/ipykernel_21211/2351571185.py:6: DtypeWarning: Columns (0: 초미세먼지 보정(㎍/㎥), 1: 미세먼지 보정(㎍/㎥)) have mixed types. Specify dtype option on import or set low_memory=False.
  df

In [37]:
# 데이터가 많아 테스트용 데이터 로드
# cold_wave_df = pd.read_csv('./data_2/한파/S-DoT_NATURE_2020.11.09-11.15.csv', encoding='euc-kr')

In [38]:
# 컬럼 공백 제거
cold_wave_df.columns = cold_wave_df.columns.str.strip()

# 서울시 데이터 뽑기
cold_wave_df = cold_wave_df[cold_wave_df['기관 명'] == '서울시']

In [39]:
# 필요한 컬럼만 뽑아오기
use_columns = ['기관 명', '등록일자', '기온(℃)']
cold_wave_df = cold_wave_df[use_columns]

# 컬럼명 변경
cold_wave_df.columns = ['agency', 'datetime', 'temp']

In [40]:
# 데이터 확인
cold_wave_df

,agency,datetime,temp
0,서울시,2022-03-07 0:07,3.5
1,서울시,2022-03-07 0:07,2.7
2,서울시,2022-03-07 0:07,-0.2
3,서울시,2022-03-07 0:07,1.3
4,서울시,2022-03-07 0:07,3.3
...,...,...,...
11670224,서울시,2021-04-11 23:07,17.2
11670225,서울시,2021-04-11 23:07,17.7
11670226,서울시,2021-04-11 23:07,18.8
11670227,서울시,2021-04-11 23:07,17.2


In [41]:
# 결측치 확인
cold_wave_df.isnull().sum()


agency           0
datetime    836951
temp         54144
dtype: int64

In [42]:

# 각 결측치 제거 및 형식 변경 (우선 제거로 진행해봄)
cold_wave_df['datetime'] = pd.to_datetime(cold_wave_df['datetime'], errors='coerce')
cold_wave_df['temp'] = pd.to_numeric(cold_wave_df['temp'], errors='coerce')
cold_wave_df = cold_wave_df.dropna(subset=['datetime', 'temp'])

# 날짜 컬럼 생성 (날짜별로 바꿔서 분석)
cold_wave_df['date'] = cold_wave_df['datetime'].dt.date

In [43]:
# 일별 최저기온 계산
daily_cold = cold_wave_df.groupby('date')['temp'].min().reset_index()

# 데이터 확인
daily_cold

,date,temp
0,2020-11-09,0.0
1,2020-11-10,0.0
2,2020-11-11,0.0
3,2020-11-12,0.0
4,2020-11-13,0.0
...,...,...
388,2022-12-24,-40.0
389,2022-12-25,-40.0
390,2022-12-26,-40.0
391,2022-12-27,-40.0


# 미세먼지

In [44]:
# 데이터 로드 인코딩 euc-kr, cp949 둘중하나 사용 (좀더 오래된 한글 인코딩 방식이 euc-kr이라고 함)
dust_df = pd.read_csv('./data/미세먼지/서울시 대기질 자료 제공_2020-2021.csv', encoding='cp949')

# 컬럼 공백 제거
dust_df.columns = dust_df.columns.str.strip()

In [45]:

# 평균값만 사용
dust_df = dust_df[dust_df['구분'] == '평균']

# 데이터 타입 변환 (errors 해석이 안되는 값들은 null 추가)
dust_df['일시'] = pd.to_datetime(dust_df['일시'], errors='coerce')

# 결측치 제거
dust_df = dust_df.dropna(subset=['일시', '미세먼지(PM10)', '초미세먼지(PM25)'])

# 컬럼명 변경
dust_df.columns = ['datetime', 'type', 'pm10', 'pm25']

# 날짜 컬럼 생성
dust_df['date'] = dust_df['datetime'].dt.date

In [46]:
# 일별 평균 계산
daily_dust = dust_df.groupby('date')[['pm10', 'pm25']].mean().reset_index()
daily_dust


,date,pm10,pm25
0,2020-01-01,32.916667,22.458333
1,2020-01-02,58.375000,39.958333
2,2020-01-03,67.166667,45.583333
3,2020-01-04,55.916667,38.166667
4,2020-01-05,51.833333,35.791667
...,...,...,...
726,2021-12-27,30.625000,17.208333
727,2021-12-28,50.250000,35.833333
728,2021-12-29,62.166667,45.000000
729,2021-12-30,28.791667,14.625000


# 폭염

In [47]:
# 데이터 로드

# 파일 경로 설정
path_heat = './data/폭염/S-DoT_NATURE_*.csv'
files = glob.glob(path_heat)
df_list = [pd.read_csv(f, encoding='cp949') for f in files]
heat_wave_df = pd.concat(df_list, ignore_index=True)


/var/folders/zh/lv2frb2s6_qbjc1lcfgty_th0000gn/T/ipykernel_21211/383167761.py:6: DtypeWarning: Columns (0: 구분, 1: 기온(℃), 2: 상대습도( %), 3: 풍향(°), 4: 풍속(m/s), 5: 돌풍 풍향(°), 6: 돌풍 풍속(m/s), 7: 조도(lux), 8: 자외선(UVI), 9: 소음(dB), 10: 진동_x(g), 11: 진동_y(g), 12: 진동_z(g), 13: 진동_x 최대(g), 14: 진동_y 최대(g), 15: 진동_z 최대(g), 16: 흑구 운도(℃), 17: 전송시간) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(f, encoding='cp949') for f in files]


In [48]:
# 서울시 데이터만 필터링
heat_wave_df = heat_wave_df[heat_wave_df['기관 명'] == '서울시']

# 컬럼명 공백 제거
heat_wave_df.columns = heat_wave_df.columns.str.strip()

# 필요한 컬럼만 추출
use_cols = ['기관 명', '등록일자', '기온(℃)']
heat_wave_df = heat_wave_df[use_cols]

# 맨뒤에 null 값만 들어간 기온 컬럼이 하나더 생성돼서 그 앞까지만 가져오기
heat_wave_df = heat_wave_df.iloc[:, [0, 1, 2]]

# 컬럼명 변경
heat_wave_df.columns = ['agency', 'datetime', 'temp']

In [49]:
# 데이터 타입 변경
heat_wave_df['datetime'] = pd.to_datetime(heat_wave_df['datetime'], errors='coerce')
heat_wave_df['temp'] = pd.to_numeric(heat_wave_df['temp'], errors='coerce')

# 결측치 제거
heat_wave_df = heat_wave_df.dropna(subset=['datetime', 'temp'])

# 이상치 제거 (공식 최고 기온: 39.6°C (2018년 8월 1일) 공식 최저 기온: -23.1°C (1927년 12월 31일) )
heat_wave_df = heat_wave_df[(heat_wave_df['temp'] > -25) & (heat_wave_df['temp'] < 45)]

# 날짜 컬럼 생성
heat_wave_df['date'] = heat_wave_df['datetime'].dt.date

# 일별 최고 기온 게산 
daily_heat = heat_wave_df.groupby('date')['temp'].max().reset_index()


In [56]:
daily_heat

,date,temp
0,2021-05-03,26.1
1,2021-05-04,21.6
2,2021-05-05,23.7
3,2021-05-06,28.3
4,2021-05-07,22.5
...,...,...
290,2022-09-28,33.5
291,2022-09-29,35.4
292,2022-09-30,35.9
293,2022-10-01,36.3


# 데이터 결합

In [68]:
print(daily_cold)
print(daily_dust)
print(daily_heat)

           date  temp
0    2020-11-09   0.0
1    2020-11-10   0.0
2    2020-11-11   0.0
3    2020-11-12   0.0
4    2020-11-13   0.0
..          ...   ...
388  2022-12-24 -40.0
389  2022-12-25 -40.0
390  2022-12-26 -40.0
391  2022-12-27 -40.0
392  2022-12-28 -40.0

[393 rows x 2 columns]
           date       pm10       pm25
0    2020-01-01  32.916667  22.458333
1    2020-01-02  58.375000  39.958333
2    2020-01-03  67.166667  45.583333
3    2020-01-04  55.916667  38.166667
4    2020-01-05  51.833333  35.791667
..          ...        ...        ...
726  2021-12-27  30.625000  17.208333
727  2021-12-28  50.250000  35.833333
728  2021-12-29  62.166667  45.000000
729  2021-12-30  28.791667  14.625000
730  2021-12-31  22.333333   6.375000

[731 rows x 3 columns]
           date  temp
0    2021-05-03  26.1
1    2021-05-04  21.6
2    2021-05-05  23.7
3    2021-05-06  28.3
4    2021-05-07  22.5
..          ...   ...
290  2022-09-28  33.5
291  2022-09-29  35.4
292  2022-09-30  35.9
293  2022-10

In [62]:
# 한파,미세먼지 데이터 병합
cold_dust_df= pd.merge(
    daily_cold,
    daily_dust
)
cold_dust_df

# 한파, 미세먼지 데이터에 폭염 데이터 병합
final_df = pd.merge(
    cold_dust_df,
    daily_heat
)

In [54]:
# 병합 후 결측값 확인
final_df

,date,temp,pm10,pm25
